In [1]:
# Cleaning the data
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

#Textblob
from textblob import TextBlob

# Load the data
response_df = pd.read_csv('../dummy_data/disaster_response_messages_trainingsv.csv')

C:\Users\Arhit\AppData\Local\Temp/ipykernel_26028/1162849292.py:20: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  response_df = pd.read_csv('../dummy_data/disaster_response_messages_trainingsv.csv')


In [2]:
response_df.head()

,id,split,message,original,genre,related,PII,request,offer,aid_related,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,12,train,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [3]:
# Perform EDA
response_df.describe()

,id,related,PII,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,21046.000000,21046.000000,21046.0,21046.000000,21046.0,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000,...,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000,21046.000000
mean,15207.883873,0.766464,0.0,0.169153,0.0,0.412667,0.078590,0.050889,0.029459,0.019101,...,0.012259,0.046945,0.278675,0.080633,0.093747,0.011356,0.094032,0.020194,0.054072,0.190488
std,8828.189299,0.441556,0.0,0.374896,0.0,0.492326,0.269104,0.219775,0.169094,0.136883,...,0.110042,0.211526,0.448358,0.272277,0.291483,0.105961,0.291880,0.140666,0.226165,0.392695
min,2.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7430.250000,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15618.500000,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22910.750000,1.000000,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,2.000000,0.0,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# Check for null values
response_df.isnull().sum()


id                            0
split                         0
message                       0
original                  12855
genre                         0
related                       0
PII                           0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops   

In [5]:
# Apply sentiment analysis to the messages
# https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

# Remove punctuation
response_df['message'] = response_df['message'].str.replace('[^\w\s]','')
response_df

C:\Users\Arhit\AppData\Local\Temp/ipykernel_26028/597793413.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  response_df['message'] = response_df['message'].str.replace('[^\w\s]','')


,id,split,message,original,genre,related,PII,request,offer,aid_related,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,Weather update a cold front from Cuba that co...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,12,train,says west side of Haiti rest of the country to...,facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,Information about the National Palace,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,The training demonstrated how to enhance micro...,NaN,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21042,30262,train,A suitable candidate has been selected and OCH...,NaN,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21043,30263,train,Proshika operating in Coxs Bazar municipality ...,NaN,news,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21044,30264,train,Some 2000 women protesting against the conduct...,NaN,news,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Remove the original column
response_df = response_df.drop(['original'], axis=1)

In [7]:
# Rmove the genre column
response_df = response_df.drop(['genre'], axis=1)


In [8]:
# NLP on the messages
# https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f

# Tokenize the messages
response_df['message'] = response_df['message'].apply(word_tokenize)
response_df

,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,"[Weather, update, a, cold, front, from, Cuba, ...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,"[Is, the, Hurricane, over, or, is, it, not, over]",1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,12,train,"[says, west, side, of, Haiti, rest, of, the, c...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,"[Information, about, the, National, Palace]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,"[Storm, at, sacred, heart, of, jesus]",1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,"[The, training, demonstrated, how, to, enhance...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21042,30262,train,"[A, suitable, candidate, has, been, selected, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21043,30263,train,"[Proshika, operating, in, Coxs, Bazar, municip...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21044,30264,train,"[Some, 2000, women, protesting, against, the, ...",1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Remove stop words
stop_words = set(stopwords.words('english'))
response_df['message'] = response_df['message'].apply(lambda x: [item for item in x if item not in stop_words])
response_df


,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,"[Weather, update, cold, front, Cuba, could, pa...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,"[Is, Hurricane]",1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,12,train,"[says, west, side, Haiti, rest, country, today...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,"[Information, National, Palace]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,"[Storm, sacred, heart, jesus]",1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,"[The, training, demonstrated, enhance, micronu...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21042,30262,train,"[A, suitable, candidate, selected, OCHA, Jakar...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21043,30263,train,"[Proshika, operating, Coxs, Bazar, municipalit...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21044,30264,train,"[Some, 2000, women, protesting, conduct, elect...",1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Lemmatize the messages
lemmatizer = WordNetLemmatizer()
response_df['message'] = response_df['message'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
response_df


,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,"[Weather, update, cold, front, Cuba, could, pa...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,"[Is, Hurricane]",1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,12,train,"[say, west, side, Haiti, rest, country, today,...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,"[Information, National, Palace]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,"[Storm, sacred, heart, jesus]",1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,"[The, training, demonstrated, enhance, micronu...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21042,30262,train,"[A, suitable, candidate, selected, OCHA, Jakar...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21043,30263,train,"[Proshika, operating, Coxs, Bazar, municipalit...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21044,30264,train,"[Some, 2000, woman, protesting, conduct, elect...",1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Convert the messages back to strings
response_df['message'] = response_df['message'].apply(lambda x: ' '.join(x))
response_df


,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,Weather update cold front Cuba could pas Haiti,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,Is Hurricane,1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,12,train,say west side Haiti rest country today tonight,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,Information National Palace,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,Storm sacred heart jesus,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,The training demonstrated enhance micronutrien...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21042,30262,train,A suitable candidate selected OCHA Jakarta cur...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21043,30263,train,Proshika operating Coxs Bazar municipality 5 u...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21044,30264,train,Some 2000 woman protesting conduct election te...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Split the data into training and testing sets
X = response_df['message']
y = response_df.drop(['message'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sentiment analysis
# https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

# Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
    

In [13]:
# Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Create two new columns
response_df['Subjectivity'] = response_df['message'].apply(getSubjectivity)
response_df['Polarity'] = response_df['message'].apply(getPolarity)

# Show the new dataframe with the new columns
response_df


,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,Subjectivity,Polarity
0,2,train,Weather update cold front Cuba could pas Haiti,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,-0.600000
1,7,train,Is Hurricane,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0.000000,0.000000
2,12,train,say west side Haiti rest country today tonight,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
3,14,train,Information National Palace,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
4,15,train,Storm sacred heart jesus,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,The training demonstrated enhance micronutrien...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21042,30262,train,A suitable candidate selected OCHA Jakarta cur...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.416667,0.183333
21043,30263,train,Proshika operating Coxs Bazar municipality 5 u...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21044,30264,train,Some 2000 woman protesting conduct election te...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000


In [14]:
# Calculate the negativity score for each message
# https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

def getNegativeScore(text):
    return TextBlob(text).sentiment.polarity

response_df['Negative Score'] = response_df['message'].apply(getNegativeScore)

response_df

,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,floods,storm,fire,earthquake,cold,other_weather,direct_report,Subjectivity,Polarity,Negative Score
0,2,train,Weather update cold front Cuba could pas Haiti,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.000000,-0.600000,-0.600000
1,7,train,Is Hurricane,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0.000000,0.000000,0.000000
2,12,train,say west side Haiti rest country today tonight,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
3,14,train,Information National Palace,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
4,15,train,Storm sacred heart jesus,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,The training demonstrated enhance micronutrien...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
21042,30262,train,A suitable candidate selected OCHA Jakarta cur...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.416667,0.183333,0.183333
21043,30263,train,Proshika operating Coxs Bazar municipality 5 u...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
21044,30264,train,Some 2000 woman protesting conduct election te...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000


In [15]:
# Create a dataframe with only negative values


In [16]:
# Sentiment analysis with tf-idf
# https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

# Assign the sentiment scores to the dataframe
response_df['Sentiment'] = response_df['Polarity'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))
response_df


,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,storm,fire,earthquake,cold,other_weather,direct_report,Subjectivity,Polarity,Negative Score,Sentiment
0,2,train,Weather update cold front Cuba could pas Haiti,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1.000000,-0.600000,-0.600000,Negative
1,7,train,Is Hurricane,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral
2,12,train,say west side Haiti rest country today tonight,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral
3,14,train,Information National Palace,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral
4,15,train,Storm sacred heart jesus,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,The training demonstrated enhance micronutrien...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral
21042,30262,train,A suitable candidate selected OCHA Jakarta cur...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.416667,0.183333,0.183333,Positive
21043,30263,train,Proshika operating Coxs Bazar municipality 5 u...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral
21044,30264,train,Some 2000 woman protesting conduct election te...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,Neutral


In [17]:
# Create a function to compute negative, neutral, and positive analysis
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

# Apply the function to the dataframe
response_df['Analysis'] = response_df['Polarity'].apply(getAnalysis)
response_df

# Show the value counts
response_df['Analysis'].value_counts()

Neutral     9568
Positive    7158
Negative    4320
Name: Analysis, dtype: int64

In [18]:
def getNegativeScore(text):
    return TextBlob(text).sentiment.polarity

# Create a new column
response_df['NegativeScore'] = response_df['message'].apply(getNegativeScore)

# Show the dataframe
response_df

,id,split,message,related,PII,request,offer,aid_related,medical_help,medical_products,...,earthquake,cold,other_weather,direct_report,Subjectivity,Polarity,Negative Score,Sentiment,Analysis,NegativeScore
0,2,train,Weather update cold front Cuba could pas Haiti,1,0,0,0,0,0,0,...,0,0,0,0,1.000000,-0.600000,-0.600000,Negative,Negative,-0.600000
1,7,train,Is Hurricane,1,0,0,0,1,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000
2,12,train,say west side Haiti rest country today tonight,1,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000
3,14,train,Information National Palace,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000
4,15,train,Storm sacred heart jesus,1,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21041,30261,train,The training demonstrated enhance micronutrien...,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000
21042,30262,train,A suitable candidate selected OCHA Jakarta cur...,0,0,0,0,0,0,0,...,0,0,0,0,0.416667,0.183333,0.183333,Positive,Positive,0.183333
21043,30263,train,Proshika operating Coxs Bazar municipality 5 u...,1,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000
21044,30264,train,Some 2000 woman protesting conduct election te...,1,0,0,0,1,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,Neutral,Neutral,0.000000


In [22]:
# Create df with only negative score
neg_df = response_df

In [19]:
# Sentiment analysis with facebook prophet
# https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

# Create a new dataframe
df = response_df[['message', 'Sentiment']]
df


,message,Sentiment
0,Weather update cold front Cuba could pas Haiti,Negative
1,Is Hurricane,Neutral
2,say west side Haiti rest country today tonight,Neutral
3,Information National Palace,Neutral
4,Storm sacred heart jesus,Neutral
...,...,...
21041,The training demonstrated enhance micronutrien...,Neutral
21042,A suitable candidate selected OCHA Jakarta cur...,Positive
21043,Proshika operating Coxs Bazar municipality 5 u...,Neutral
21044,Some 2000 woman protesting conduct election te...,Neutral


In [20]:
# Rename the columns
df = df.rename(columns={'message': 'ds', 'Sentiment': 'y'})
df

,ds,y
0,Weather update cold front Cuba could pas Haiti,Negative
1,Is Hurricane,Neutral
2,say west side Haiti rest country today tonight,Neutral
3,Information National Palace,Neutral
4,Storm sacred heart jesus,Neutral
...,...,...
21041,The training demonstrated enhance micronutrien...,Neutral
21042,A suitable candidate selected OCHA Jakarta cur...,Positive
21043,Proshika operating Coxs Bazar municipality 5 u...,Neutral
21044,Some 2000 woman protesting conduct election te...,Neutral


In [21]:
# Create a new dataframe
df2 = response_df[['message', 'Analysis']]
df2

,message,Analysis
0,Weather update cold front Cuba could pas Haiti,Negative
1,Is Hurricane,Neutral
2,say west side Haiti rest country today tonight,Neutral
3,Information National Palace,Neutral
4,Storm sacred heart jesus,Neutral
...,...,...
21041,The training demonstrated enhance micronutrien...,Neutral
21042,A suitable candidate selected OCHA Jakarta cur...,Positive
21043,Proshika operating Coxs Bazar municipality 5 u...,Neutral
21044,Some 2000 woman protesting conduct election te...,Neutral
